# Multiprocessing module

Bets suited for single machine multicore parallelism.
Great for **CPU bound issues.**

**Treads** vs multiple **Processes**: multiple treads still suffer from the python GIL and so wont get you far, except when using numpy, which does not suffer from the GIL because it is in C. But multi-processing will spin up multiple python interpreters that each have their own GIL. 

In [1]:
import random
import multiprocessing
from joblib import Parallel, delayed

def monte_carlo_pi_part(n):
    """
    Function to estimate pi using Monte Carlo method in parallel.
    n is the number of random points to generate.
    """
    count = sum(1 for _ in range(n) if random.random()**2 + random.random()**2 <= 1)
    return count

def main(num_samples):
    inside_circle = monte_carlo_pi_part(num_samples)
    pi_estimate = 4 * inside_circle / num_samples  # Estimate Pi
    print(f"Estimated Pi = {pi_estimate}")


def main_multi_process(processes, total_points):
    with multiprocessing.Pool(processes=processes) as pool:
        # Split total_points among processes
        part_counts = [total_points // processes for _ in range(processes)]
        
        # Estimate pi using all processes
        counts = pool.map(monte_carlo_pi_part, part_counts)
        
        # Combine the counts from all processes
        total_count = sum(counts)
        
        # Estimate of pi
        pi_estimate = 4 * total_count / total_points
        print(f"Estimated Pi = {pi_estimate}")
        

def main_joblib(processes, total_points):
    # Use joblib to parallelize the Monte Carlo simulation across n_jobs processes
    results = Parallel(n_jobs=processes)(delayed(monte_carlo_pi_part)(total_points // processes) for _ in range(processes))
    
    # Combine the results from all processes
    total_count = sum(results)
    
    # Estimate of pi
    pi_estimate = 4 * total_count / total_points
    print(f"Estimated Pi = {pi_estimate}")

In [2]:
multiprocessing.cpu_count()

4

In [3]:
PROCESSES = multiprocessing.cpu_count() # Number of processes to use
TOTAL_POINTS = 10_000_000 # Total number of points to generate

In [4]:
%%timeit

main(TOTAL_POINTS)

Estimated Pi = 3.141016
Estimated Pi = 3.1426616
Estimated Pi = 3.1411588
Estimated Pi = 3.1412756
Estimated Pi = 3.1414144
Estimated Pi = 3.1422808
Estimated Pi = 3.1417344
Estimated Pi = 3.1402864
3.53 s ± 22.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit

main_multi_process(PROCESSES, TOTAL_POINTS)

Estimated Pi = 3.1419708
Estimated Pi = 3.1416924
Estimated Pi = 3.1422224
Estimated Pi = 3.1416016
Estimated Pi = 3.141178
Estimated Pi = 3.1409372
Estimated Pi = 3.141686
Estimated Pi = 3.1415552
1.8 s ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit

main_joblib(PROCESSES, TOTAL_POINTS)

Estimated Pi = 3.1420292
Estimated Pi = 3.1409736
Estimated Pi = 3.1418224
Estimated Pi = 3.140754
Estimated Pi = 3.1419864
Estimated Pi = 3.1419228
Estimated Pi = 3.1407304
Estimated Pi = 3.1416512
1.79 s ± 18.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
